<a href="https://colab.research.google.com/github/bhausaheb-pachaptil/Task/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Input

# Dummy sequential data (100 samples, 10 timesteps, 5 features)
X = np.random.rand(100, 10, 5)
y = np.random.randint(0, 2, 100)

# RNN model
model = Sequential([
    Input(shape=(10, 5)), # Recommended way to specify input shape in Sequential models
    SimpleRNN(16),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=5, batch_size=8)

Using Colab cache for faster access to the 'heartbeat' dataset.
Path to dataset files: /kaggle/input/heartbeat
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5343 - loss: 0.7201
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4944 - loss: 0.7188
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5292 - loss: 0.7026
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5182 - loss: 0.6984
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5065 - loss: 0.7075


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shayanfazeli/heartbeat")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'heartbeat' dataset.
Path to dataset files: /kaggle/input/heartbeat


In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Input, Dropout

# ============================
# 1) Set dataset path
# ============================

import kagglehub
path = kagglehub.dataset_download("shayanfazeli/heartbeat")
print("Dataset path:", path)

train_path = os.path.join(path, "mitbih_train.csv")
test_path  = os.path.join(path, "mitbih_test.csv")

print("Train file:", train_path)
print("Test file:", test_path)

# ============================
# 2) Load data
# ============================

train_df = pd.read_csv(train_path, header=None)
test_df  = pd.read_csv(test_path, header=None)

print("Train shape:", train_df.shape)  # (N_train, 188)
print("Test shape:", test_df.shape)    # (N_test, 188)

# Last column = label, first 187 = signal
X_train = train_df.iloc[:, :-1].values   # (N_train, 187)
y_train = train_df.iloc[:, -1].values    # (N_train,)

X_test  = test_df.iloc[:, :-1].values
y_test  = test_df.iloc[:, -1].values

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Classes:", np.unique(y_train))

# ============================
# 3) Normalize & reshape for RNN
# ============================

# Normalize each sample to [-1,1] by dividing by max abs value per sample
train_max = np.max(np.abs(X_train), axis=1, keepdims=True)
test_max  = np.max(np.abs(X_test), axis=1, keepdims=True)

# Avoid division by zero
train_max[train_max == 0] = 1.0
test_max[test_max == 0] = 1.0

X_train_norm = X_train / train_max
X_test_norm  = X_test / test_max

# RNN expects shape (samples, timesteps, features)
# Here: timesteps = 187, features = 1
X_train_rnn = X_train_norm.reshape((-1, 187, 1))
X_test_rnn  = X_test_norm.reshape((-1, 187, 1))

print("X_train_rnn shape:", X_train_rnn.shape)
print("X_test_rnn shape:", X_test_rnn.shape)

num_classes = len(np.unique(y_train))

# ============================
# 4) Class weights (dataset is imbalanced)
# ============================

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weight_dict = {cls: w for cls, w in zip(np.unique(y_train), class_weights)}
print("Class weights:", class_weight_dict)

# ============================
# 5) Build RNN model
# ============================

model = Sequential([
    Input(shape=(187, 1)),
    SimpleRNN(64, return_sequences=True),
    Dropout(0.3),
    SimpleRNN(32),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ============================
# 6) Train model
# ============================

history = model.fit(
    X_train_rnn, y_train,
    validation_split=0.2,
    epochs=10,            # increase later if needed
    batch_size=128,
    class_weight=class_weight_dict,
    verbose=1
)

# ============================
# 7) Evaluate on test set
# ============================

test_loss, test_acc = model.evaluate(X_test_rnn, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.4f}")

# Predictions for metrics
y_pred_probs = model.predict(X_test_rnn, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification report:")
print(classification_report(y_test, y_pred))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))


100%|██████████| 98.8M/98.8M [00:05<00:00, 19.1MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/shayanfazeli/heartbeat/versions/1
Train file: /root/.cache/kagglehub/datasets/shayanfazeli/heartbeat/versions/1/mitbih_train.csv
Test file: /root/.cache/kagglehub/datasets/shayanfazeli/heartbeat/versions/1/mitbih_test.csv
Train shape: (87554, 188)
Test shape: (21892, 188)
X_train shape: (87554, 187)
y_train shape: (87554,)
Classes: [0. 1. 2. 3. 4.]
X_train_rnn shape: (87554, 187, 1)
X_test_rnn shape: (21892, 187, 1)
Class weights: {np.float64(0.0): np.float64(0.24162492583240192), np.float64(1.0): np.float64(7.877103013945119), np.float64(2.0): np.float64(3.0253628196268143), np.float64(3.0): np.float64(27.317940717628705), np.float64(4.0): np.float64(2.7228735810915876)}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 187, 64)        │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 187, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,549 (33.39 KB)

 Trainable params: 8,549 (33.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 26s 37ms/step - accuracy: 0.9813 - loss: 0.0238 - val_accuracy: 0.1387 - val_loss: 10.6708
Epoch 2/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 1.0000 - loss: 2.0625e-05 - val_accuracy: 0.1387 - val_loss: 12.3259
Epoch 3/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 1.0000 - loss: 5.6470e-06 - val_accuracy: 0.1387 - val_loss: 13.3015
Epoch 4/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 1.0000 - loss: 2.5397e-06 - val_accuracy: 0.1387 - val_loss: 14.0449
Epoch 5/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 1.0000 - loss: 1.4101e-06 - val_accuracy: 0.1387 - val_loss: 14.6609
Epoch 6/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 1.0000 - loss: 8.0899e-07 - val_accuracy: 0.1387 - val_loss: 15.1889
Epoch 7/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 1.0000 - loss: 5.3891e-07 - val_accuracy: 0.1387 - val_loss: 15.6826
Epoch 8/10
548/548 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - acc

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
